In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shusrith/imbd-dataset-tokenized")

print("Path to dataset files:", path)

100%|██████████| 18.9M/18.9M [00:01<00:00, 10.0MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shusrith/imbd-dataset-tokenized/versions/4


In [24]:
import pandas as pd
df = pd.read_csv("data.csv")
df

,review,sentiment
0,"[101, 2028, 1997, 1996, 2060, 15814, 2038, 385...",1
1,"[3632, 2073, 2060, 3065, 2876, 2102, 8108, 529...",1
2,"[101, 1037, 6919, 2210, 2537, 7987, 7987, 1996...",1
3,"[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126...",1
4,"[101, 10468, 2045, 2015, 1037, 2155, 2073, 103...",0
...,...,...
86861,"[101, 1045, 2572, 1037, 3234, 4036, 1999, 2877...",0
86862,"[10760, 3268, 1997, 1996, 11113, 15613, 10834,...",0
86863,"[101, 10047, 2183, 2000, 2031, 2000, 21090, 20...",0
86864,"[2011, 2028, 2004, 1996, 2466, 11320, 11140, 2...",0


In [25]:
import ast
df["review"] = df["review"].apply(lambda x: ast.literal_eval(x))
type(df.iloc[0, 0])

list

In [54]:
import torch.nn as nn
import torch

class Model(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x, h=None):
        x = self.embedding(x)
        x, h = self.lstm(x, h)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [55]:
model = Model(64, 128, 30522)

In [56]:
import torch.optim as optim

batch_size = 1024
seq_len = 200
epochs = 10
learning_rate = 0.001

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [57]:
from torch.utils.data import DataLoader, TensorDataset
train = TensorDataset(
    torch.tensor(df["review"][:50000].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][:50000].tolist(), dtype=torch.float32),
)
test = TensorDataset(
    torch.tensor(df["review"][50000:].tolist(), dtype=torch.long),
    torch.tensor(df["sentiment"][50000:].tolist(), dtype=torch.float32),
)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

In [58]:
vocab_size = 30522
hidden_dim = 128

In [60]:
from tqdm import tqdm

model = model.to("cuda")
for epoch in range(epochs):
    model.train()
    train_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")

    total_train_loss = 0

    for train_data, train_labels in train_tqdm:
        train_data, train_labels = train_data.to("cuda"), train_labels.to("cuda")

        batch_size = train_data.size(0)
        # h = torch.zeros(1, batch_size, hidden_dim).to("cuda")
        # print(h.shape)
        optimizer.zero_grad()
        output = model(train_data)  
        h = h.detach()
        output = output.squeeze()
        loss = criterion(output, train_labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        train_tqdm.set_postfix(loss=loss.item())

    avg_train_loss = total_train_loss / len(train_loader)

    model.eval()
    val_tqdm = tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Val]")

    total_val_loss = 0
    with torch.no_grad():
        for val_data, val_labels in val_tqdm:
            val_data, val_labels = val_data.to("cuda"), val_labels.to("cuda")

            batch_size = val_data.size(0)
            # h = torch.zeros(1, batch_size, hidden_dim).to("cuda")  

            output = model(val_data)

            output = output.squeeze()
            loss = criterion(output, val_labels)
            total_val_loss += loss.item()

            val_tqdm.set_postfix(loss=loss.item())

    avg_val_loss = total_val_loss / len(test_loader)

    print(
        f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}"
    )

Epoch 1/10 [Val]: 100%|██████████| 37/37 [00:00<00:00, 39.14it/s, loss=0.613]


Epoch 1/10 | Train Loss: 0.6873 | Val Loss: 0.6760


Epoch 2/10 [Val]: 100%|██████████| 37/37 [00:00<00:00, 39.30it/s, loss=0.68] 


Epoch 2/10 | Train Loss: 0.6827 | Val Loss: 0.6899


Epoch 3/10 [Val]: 100%|██████████| 37/37 [00:01<00:00, 31.35it/s, loss=0.689]


...

Epoch 8/10 | Train Loss: 0.6782 | Val Loss: 0.6848


Epoch 9/10 [Val]: 100%|██████████| 37/37 [00:00<00:00, 38.66it/s, loss=0.556]


Epoch 9/10 | Train Loss: 0.6734 | Val Loss: 0.6968


Epoch 10/10 [Val]: 100%|██████████| 37/37 [00:00<00:00, 38.14it/s, loss=0.704]

Epoch 10/10 | Train Loss: 0.6813 | Val Loss: 0.6875


In [35]:
torch.nn.functional.sigmoid(output)

tensor([0.6199, 0.3544], device='cuda:0')

In [61]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
import numpy as np

# Set parameters
vocab_size = 10000  # Number of words to keep based on frequency
max_length = 200  # Max length of a review
embedding_dim = 128  # Embedding vector size
lstm_units = 64  # Number of LSTM units
batch_size = 64
epochs = 5

# Load IMDB dataset
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure uniform length
x_train = pad_sequences(x_train, maxlen=max_length, padding="post", truncating="post")
x_test = pad_sequences(x_test, maxlen=max_length, padding="post", truncating="post")

# Build the LSTM model
model = keras.Sequential(
    [
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.LSTM(lstm_units, return_sequences=True),
        keras.layers.LSTM(lstm_units),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dense(
            1, activation="sigmoid"
        ),  # Binary classification (positive/negative)
    ]
)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

2025-03-06 06:21:37.338819: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-06 06:21:37.433953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-06 06:21:37.472817: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-06 06:21:37.485991: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-06 06:21:37.559474: I tensorflow/core/platform/cpu_feature_guar

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/5


/home/shusrith/projects/torch/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1741222305.679446    6351 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-03-06 06:21:45.682150: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


391/391 ━━━━━━━━━━━━━━━━━━━━ 52s 127ms/step - accuracy: 0.5541 - loss: 0.6772 - val_accuracy: 0.5771 - val_loss: 0.6544
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 55s 140ms/step - accuracy: 0.5220 - loss: 0.6883 - val_accuracy: 0.5375 - val_loss: 0.6902
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 54s 137ms/step - accuracy: 0.5419 - loss: 0.6867 - val_accuracy: 0.5820 - val_loss: 0.6738
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 51s 130ms/step - accuracy: 0.6780 - loss: 0.5948 - val_accuracy: 0.7869 - val_loss: 0.4881
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 56s 143ms/step - accuracy: 0.8356 - loss: 0.3868 - val_accuracy: 0.8398 - val_loss: 0.3737
782/782 ━━━━━━━━━━━━━━━━━━━━ 17s 22ms/step - accuracy: 0.8413 - loss: 0.3729
Test Accuracy: 0.8398
